## CBoWモデルのトレーニング

このノートブックは[AI for Beginners Curriculum](http://aka.ms/ai-beginners)の一部です。

この例では、CBoW言語モデルをトレーニングして独自のWord2Vec埋め込み空間を作成する方法を見ていきます。テキストのソースとしてAG Newsデータセットを使用します。


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

まず、データセットを読み込み、トークナイザーとボキャブラリーを定義しましょう。計算量を少し制限するために、`vocab_size` を5000に設定します。


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoWモデル

CBoWは、$2N$個の周辺単語から1つの単語を予測することを学習します。例えば、$N=1$の場合、文 *I like to train networks* から以下のペアが得られます: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks)。ここで、最初の単語は入力として使用される周辺単語で、2番目の単語が予測対象の単語です。

次の単語を予測するネットワークを構築するには、周辺単語を入力として与え、単語番号を出力として得る必要があります。CBoWネットワークのアーキテクチャは以下の通りです:

* 入力単語は埋め込み層を通過します。この埋め込み層が私たちのWord2Vec埋め込みとなり、`embedder`変数として個別に定義します。この例では埋め込みサイズを30に設定しますが、より高次元で実験することも可能です（実際のWord2Vecでは300次元です）。
* 埋め込みベクトルは、その後、出力単語を予測する線形層に渡されます。この層には`vocab_size`個のニューロンがあります。

出力に関しては、損失関数として`CrossEntropyLoss`を使用する場合、ワンホットエンコーディングを行わずに、単語番号をそのまま期待される結果として提供する必要があります。


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## トレーニングデータの準備

それでは、テキストからCBoWの単語ペアを計算するメイン関数をプログラムしてみましょう。この関数ではウィンドウサイズを指定でき、入力単語と出力単語のペアセットを返します。この関数は単語だけでなく、ベクトルやテンソルにも使用できる点に注意してください。これにより、テキストをエンコードしてから`to_cbow`関数に渡すことが可能になります。


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


トレーニングデータセットを準備しましょう。すべてのニュースを確認し、`to_cbow` を呼び出して単語ペアのリストを取得し、そのペアを `X` と `Y` に追加します。時間を節約するために、最初の10k件のニュース項目のみを考慮しますが、待つ時間がある場合やより良い埋め込みを得たい場合は、この制限を簡単に取り除くことができます :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

そのデータを1つのデータセットに変換し、データローダーを作成します。


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

そのデータを1つのデータセットに変換し、データローダーを作成します。


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

では、実際のトレーニングを始めましょう。学習率がかなり高い`SGD`オプティマイザーを使用します。また、`Adam`などの他のオプティマイザーを試してみることもできます。まずは10エポックでトレーニングを行いますが、さらに損失を減らしたい場合はこのセルを再実行することができます。


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vecを試してみる

Word2Vecを使用するために、語彙内のすべての単語に対応するベクトルを抽出してみましょう:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

例えば、単語**Paris**がどのようにベクトルにエンコードされるか見てみましょう。


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Word2Vecを使用して類義語を探すのは興味深いです。以下の関数は、指定された入力に対して最も近い`n`個の単語を返します。それらを見つけるために、$|w_i - v|$のノルムを計算します。ここで、$v$は入力単語に対応するベクトルであり、$w_i$は語彙内の$i$番目の単語のエンコーディングです。その後、配列をソートし、`argsort`を使用して対応するインデックスを返し、語彙内で最も近い単語の位置をエンコードするリストの最初の`n`個の要素を取得します。


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## 要点

CBoWのような巧妙な手法を使うことで、Word2Vecモデルを訓練することができます。また、中心の単語を与えられたときに隣接する単語を予測するように訓練されたskip-gramモデルを試してみて、その性能を確認してみるのも良いでしょう。



---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてご参照ください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は責任を負いません。
